In [1]:
import pinecone
from pinecone import ServerlessSpec, Pinecone
import os
import time
from dotenv import load_dotenv
import pandas as pd
from sentence_transformers import SentenceTransformer

load_dotenv()

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY environment variable not set")

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Create an index
index_name = "movies-actors"
if index_name not in pc.list_indexes().names():  # Corrected index name check
    pc.create_index(
    name=index_name,
    vector_type="dense",
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ),
    deletion_protection="disabled",
    tags={
        "environment": "development"
    }
)
    while index_name not in pc.list_indexes().names(): #wait until index is created.
        time.sleep(1)



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Connect to the index
index = pc.Index(index_name)
# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [7]:
query = model.encode("died in 1980").tolist()

response = index.query(
    vector=query,
    top_k=5,  # Retrieve top 5 similar movies
    include_metadata=True
)

print(response)

{'matches': [{'id': 'nm0085481',
              'metadata': {'knownFor': 'tt0099260', 'name': 'Rip Black'},
              'score': 0.408161283,
              'values': []},
             {'id': 'nm0063610',
              'metadata': {'knownFor': '\\N', 'name': 'Ethel Beal'},
              'score': 0.373275131,
              'values': []},
             {'id': 'nm0091328',
              'metadata': {'knownFor': 'tt0117554', 'name': 'Leroy Boerger'},
              'score': 0.372418225,
              'values': []},
             {'id': 'nm0041372',
              'metadata': {'knownFor': 'tt0041710,tt0044025,tt0140975',
                           'name': 'Robert Auboyneau'},
              'score': 0.364476055,
              'values': []},
             {'id': 'nm0020759',
              'metadata': {'knownFor': 'tt0142192,tt0167837,tt0216772,tt0279721',
                           'name': 'LeRoy Allen'},
              'score': 0.356951088,
              'values': []}],
 'namespace': '',
 'usage':